In [3]:
#Importare biblioteci necesare

import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pickle
import ipyplot
from sklearn.model_selection import train_test_split

In [4]:
#Importare poze + Atribuire etichete + Redimensionare

data_dir = './data2/'

categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
data = []

def make_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        label = categories.index(category)
        
        for img_name in os.listdir(path):
            image_path = os.path.join(path, img_name)
            image = cv2.imread(image_path)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # BGR -> Gray
            image = cv2.resize(image, (300, 200)) # redimensionare
            image = np.array(image, dtype = np.float32) #img -> array
            
            data.append([image, label])
            
    np.random.shuffle(data)
    
    features = []
    labels = []
    
    for img, lbl in data:
        features.append(img)
        labels.append(lbl)
        
    feature = np.array(features, dtype = np.float32)
    labels = np.array(labels, dtype = np.float32)
    
    return [features, labels]

        

 
  

In [5]:
#Features - Labels - etichetele pozelor

[features, labels] = make_data()

In [6]:
print(np.shape(features))
print(np.shape(labels))

(2416, 200, 300, 3)
(2416,)


In [7]:
#Impartire Train - Test

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size = 0.2)

#Intensitatile pixelilor din poze
print(np.shape(x_train))
print(np.shape(x_test))

#Etichetele
print(np.shape(y_train))
print(np.shape(y_test))

(1932, 200, 300, 3)
(484, 200, 300, 3)
(1932,)
(484,)


In [8]:
#Flat

x_train_flat = []
for i in x_train:
    x_train_flat.append(i.flatten())
    
x_test_flat = []
for i in x_test:
    x_test_flat.append(i.flatten())

print(np.shape(x_train_flat))
print(np.shape(x_test_flat))

(1932, 180000)
(484, 180000)


In [9]:
from sklearn.decomposition import PCA

pca = PCA(n_components=400)
pca.fit(x_train_flat)

pca1 = PCA(n_components=400)
pca1.fit(x_test_flat)

PCA(n_components=400)

In [10]:
x_train_pca = pca.transform(x_train_flat)
print(np.shape(x_train_pca))

x_test_pca = pca.transform(x_test_flat)
print(np.shape(x_test_pca))

(1932, 400)
(484, 400)


In [11]:
from sklearn import svm

clf = svm.SVC()
clf.fit(x_train_pca, y_train)

SVC()

In [12]:
test_predict = clf.predict(x_test_pca)


In [13]:
import collections

print(collections.Counter(y_test))
print("\n\n")
print(collections.Counter(test_predict))

Counter({4.0: 131, 3.0: 103, 2.0: 101, 0.0: 96, 1.0: 53})



Counter({4.0: 129, 3.0: 114, 0.0: 101, 2.0: 86, 1.0: 54})


In [14]:
cnt = 0
for i in range(len(y_test)):
    if test_predict[i] == y_test[i]:
        cnt += 1
print ("Clasificari corecte : " + str(cnt) + " / " + str(len(y_test)))  
print ("Rata este : " + str(cnt / len(y_test) * 100))

Clasificari corecte : 307 / 484
Rata este : 63.42975206611571


In [15]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true = y_test[:] ,y_pred = test_predict)
print(cm)

[[64 10  4  3 15]
 [13 34  1  1  4]
 [ 8  4 53  4 32]
 [ 2  0  1 89 11]
 [14  6 27 17 67]]
